In [1]:
#!g1.1
import torch
import torch.nn as nn
from torchvision.datasets import MNIST
import torchvision.transforms as T

from torch.utils.data import DataLoader
from IPython.display import clear_output
from tqdm import tqdm
import matplotlib.pyplot as plt
from torch.optim import Adam
import numpy as np
from PIL import Image
import math

OSError: [WinError 126] Не найден указанный модуль. Error loading "C:\Users\Vica\AppData\Roaming\Python\Python311\site-packages\torch\lib\shm.dll" or one of its dependencies.

In [16]:
#!g1.1
mnist_transforms = T.Compose(
    [
        T.Resize((64, 64)),
        T.ToTensor(),
    ]
)

train_dataset = MNIST('mnist', train=True, transform=mnist_transforms, download=True)
valid_dataset = MNIST('mnist', train=False, transform=mnist_transforms, download=True)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=8, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=False, num_workers=8, pin_memory=True)

Обучите автоэнкодер на датасете MNIST, постарайтесь сделать его с размером эмбеддинга не больше 128. Сгенерируйте эмбеддинги для тысячи объектов из обучающей выборки (как в семинаре), обучите на этих эмбеддингах Случайный лес. Добейтесь Accuracy больше 90%.

Сделайте эмбеддинги для 1000 объектов из обучающей выборки, тензор с ними должен иметь размерность (1000,N), где N≤128. Мы обучим на этом тензоре Случайный лес из библиотеки scikit-learn, а именно RandomForestClassifier(random_state=0). После чего проверим качество на ваших эмбеддингах для тестовой выборки, сдайте их тоже. Тензор с эмбеддингами для тестовой выборки должен иметь размерность (10000,N). Accuracy нашего обученного случайного леса должна быть больше 90% на тестовых эмбеддингах.

Чтобы корректно сдать все эмбеддинги воспользуйтесь этой функцией:

In [3]:
#!g1.1
def train(model):
    model.train()

    train_loss = 0

    for x, _ in tqdm(train_loader, desc='Train'):
        x = x.to(device)

        optimizer.zero_grad()

        output = model(x)

        loss = loss_fn(output, x)

        train_loss += loss.item()

        loss.backward()

        optimizer.step()

    train_loss /= len(train_loader)

    return train_loss

In [4]:
#!g1.1
@torch.inference_mode()
def evaluate(model, loader):
    model.eval()

    total_loss = 0

    for x, _ in tqdm(loader, desc='Evaluation'):
        x = x.to(device)

        output = model(x)

        loss = loss_fn(output, x)

        total_loss += loss.item()

    total_loss /= len(loader)

    return total_loss


In [5]:
#!g1.1
def plot_stats(
        train_loss: list[float],
        valid_loss: list[float],
        title: str
):
    plt.figure(figsize=(16, 8))

    plt.title(title + ' loss')

    plt.plot(train_loss, label='Train loss')
    plt.plot(valid_loss, label='Valid loss')
    plt.legend()
    plt.grid()

    plt.show()


In [6]:
#!g1.1
from torchvision.utils import make_grid


@torch.inference_mode()
def visualize(model, xs):
    model.eval()

    to_pil = T.ToPILImage()

    outputs = model(xs.to(device)).cpu()

    plt.figure(figsize=(20, 4))

    plt.imshow(
        to_pil(
            torch.cat(
                (
                    make_grid(xs[:10], nrow=10, pad_value=1),
                    make_grid(outputs[:10], nrow=10, pad_value=1)
                ),
                dim=1
            )
        )
    )

    plt.axis('off')

    plt.show()

In [7]:
#!g1.1
def whole_train_valid_cycle(model, num_epochs, title):
    train_loss_history, valid_loss_history = [], []

    for epoch in range(num_epochs):
        train_loss = train(model)
        valid_loss = evaluate(model, valid_loader)

        train_loss_history.append(train_loss)
        valid_loss_history.append(valid_loss)

        clear_output(wait=True)

        visualize(model, next(iter(valid_loader))[0])

        plot_stats(
            train_loss_history, valid_loss_history,
            title
        )


In [8]:
class Block(nn.Module):
    def __init__(self, in_channels: int, out_channels: int, kernel_size: int, stride: int = 1, upsample: bool = False):
        super().__init__()
        self.upsample = upsample

        self.conv = nn.Conv2d(
            in_channels=in_channels,
            out_channels=out_channels,
            kernel_size=kernel_size,
            stride=stride,
            padding=1,
            bias=False
        )
        self.norm = nn.BatchNorm2d(out_channels)
        self.act = nn.LeakyReLU(0.2)

    def forward(self, x):
        if self.upsample:
            x = nn.functional.interpolate(x, scale_factor=2, mode='bilinear', align_corners=False, recompute_scale_factor=False)

        return self.act(self.norm(self.conv(x)))


class DenoisingBlock(Block):
    def __init__(self, in_channels: int, out_channels: int, kernel_size: int, stride: int = 1, upsample: bool = False):
        super().__init__(in_channels, out_channels, kernel_size, stride, upsample)

        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        if self.upsample:
            x = nn.functional.interpolate(x, scale_factor=2, mode='bilinear', align_corners=False, recompute_scale_factor=False)

        if self.training:
            x = x + torch.randn_like(x) * 0.05

        return self.act(self.norm(self.conv(x)))


class AutoEncoder(nn.Module):
    def __init__(self, in_channels: int, base_block: nn.Module = Block, base_size: int = 32, num_blocks: int = 4):
        super().__init__()

        self.base_size = base_size

        # encoder creation

        encoder_blocks = []

        for i in range(num_blocks):
            encoder_blocks.append(
                base_block(
                    in_channels=base_size if i else in_channels,
                    out_channels=base_size,
                    kernel_size=3,
                    stride=2
                )
            )

        encoder_blocks.append(
            base_block(
                in_channels=base_size,
                out_channels=base_size,
                kernel_size=3
            ).conv
        )

        self.encoder = nn.Sequential(*encoder_blocks)  # N -> N // (2 ** num_blocks)

        # decoder creation

        decoder_blocks = []

        for i in range(num_blocks):
            decoder_blocks.append(
                base_block(
                    in_channels=base_size,
                    out_channels=base_size,
                    kernel_size=3,
                    upsample=True
                )
            )

        decoder_blocks.append(
            base_block(
                in_channels=base_size,
                out_channels=in_channels,
                kernel_size=3
            ).conv
        )
        decoder_blocks.append(nn.Sigmoid())

        self.decoder = nn.Sequential(*decoder_blocks)  # N // (2 ** num_blocks) -> N

        self.flatten = nn.Flatten()

    def forward(self, x):
        # x.shape = [bs, in_channels, N, N]
        x = self.encoder(x)
        # x.shape = [bs, base_size, N // (2 ** num_blocks), N // (2 ** num_blocks)]
        x = self.decoder(x)
        # x.shape = [bs, in_channels, N, N]

        return x

    @torch.inference_mode()
    def encode(self, x):
        # x.shape = [bs, in_channels, N, N]
        return self.flatten(self.encoder(x)) # output.shape = [bs, base_size * N ** 2 / (2 ** num_blocks) ** 2]

    @torch.inference_mode()
    def decode(self, x):
        # x.shape = [bs, base_size * N ** 2 / (2 ** num_blocks) ** 2]
        latent_size = int(math.sqrt(x.shape[1] // self.base_size))

        return self.decoder(x.view(-1, self.base_size, latent_size, latent_size))  # output.shape = [bs, in_channels, N, N]


class DenoisingAutoEncoder(AutoEncoder):
    def __init__(self, in_channels: int, base_block: nn.Module = DenoisingBlock, base_size: int = 32, num_blocks: int = 4):
        super().__init__(in_channels, base_block, base_size, num_blocks)

    def forward(self, x):
        if self.training:
            x = torch.clip(x + torch.randn_like(x) * 0.1, min=0, max=1)

        # x.shape = [bs, in_channels, N, N]
        x = self.encoder(x)
        # x.shape = [bs, base_size, N // (2 ** num_blocks), N // (2 ** num_blocks)]
        x = self.decoder(x)
        # x.shape = [bs, in_channels, N, N]

        return x

In [9]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(device)
print(torch.cuda.get_device_name())

loss_fn = nn.MSELoss()

cuda:0
NVIDIA GeForce RTX 3050 Laptop GPU


In [10]:
model = autoencoder = DenoisingAutoEncoder(in_channels=1, base_size=128, num_blocks=4).to(device)
optimizer = Adam(model.parameters(), lr=1e-3)

In [11]:
num_epochs = 10
for epoch in range(num_epochs):
    for data in train_loader:
        img, _ = data
        img = img.to(device)
        optimizer.zero_grad()
        output = model(img)
        loss = loss_fn(output, img)
        loss.backward()
        optimizer.step()
    print('hi')

embeddings = []
targets = []
with torch.no_grad():
    for data in train_loader:
        img, target = data
        img = img.to(device)
        embedding = model.encode(img)
        embeddings.append(embedding.cpu().numpy())
        targets.append(target.numpy())

embeddings = np.concatenate(embeddings)
targets = np.concatenate(targets)

hi
hi
hi


KeyboardInterrupt: 

In [18]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

autoencoder = DenoisingAutoEncoder(in_channels=1, base_size=128, num_blocks=4)
autoencoder.to(device)
autoencoder.eval()  # переводим модель в режим оценки (не тренировки)

# Получаем эмбеддинги для обучающей выборки
train_embeddings = []
train_targets = []
with torch.no_grad():
    for data in train_loader:
        img, target = data
        img = img.to(device)
        embedding = autoencoder.encode(img).cpu().numpy()
        train_embeddings.append(embedding)
        train_targets.append(target.numpy())

train_embeddings = torch.tensor(train_embeddings).view(-1, 128)
train_targets = torch.tensor(train_targets).view(-1)

train_embeddings = torch.cat(train_embeddings, dim=0)
train_targets = torch.cat(train_targets, dim=0)

# Обучение Random Forest на эмбеддингах обучающей выборки
clf = RandomForestClassifier(random_state=0)
clf.fit(train_embeddings[:1000], train_targets[:1000])


test_embeddings = []
test_targets = []
with torch.no_grad():
    for data in valid_loader:
        img, target = data
        img = img.to(device)
        embedding = autoencoder.encode(img).cpu().numpy()
        test_embeddings.append(embedding)
        test_targets.append(target.numpy())

test_embeddings = torch.tensor(test_embeddings).view(-1, 128)  # Преобразуем в тензор и выравниваем размерность
test_targets = torch.tensor(test_targets).view(-1)  # Преобразуем в тензор и выравниваем размерность

test_preds = clf.predict(test_embeddings)
accuracy = accuracy_score(test_targets, test_preds)
print(f"Accuracy on test set: {accuracy * 100:.2f}%")

ValueError: expected sequence of length 64 at dim 1 (got 32)

In [16]:
# torch.save(model.state_dict(), 'model.pth')

In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [14]:
clf = RandomForestClassifier(random_state=0)
clf.fit(embeddings[:1000], targets[:1000])
test_embeddings = []
test_targets = []
with torch.no_grad():
    for data in valid_loader:
        img, target = data
        img = img.to(device)
        embedding = autoencoder.encode(img)
        test_embeddings.append(embedding.cpu().numpy())
        test_targets.append(target.numpy())

test_embeddings = np.concatenate(test_embeddings)
test_targets = np.concatenate(test_targets)

In [15]:
test_preds = clf.predict(test_embeddings)
accuracy = accuracy_score(test_targets, test_preds)
print(f"Accuracy on test set: {accuracy * 100:.2f}%")

Accuracy on test set: 91.91%


In [49]:
embeddings.shape[0]

60000

In [42]:
def save_embeddings(x_train, y_train, x_valid, y_valid):
    assert x_train.shape[0] == 1000
    assert x_valid.shape[0] == 10000
    
    assert y_train.shape[0] == 1000
    assert y_valid.shape[0] == 10000

    torch.save(
        {
            'x_train': x_train,
            'y_train': y_train,
            'x_valid': x_valid,
            'y_valid': y_valid
        },
        'embeddings3.pt'
    )

In [43]:
save_embeddings(embeddings[:1000], targets[:1000], test_embeddings, test_targets)